In [1]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.auth import TokenAuthentication
from codeflare_sdk.job.jobs import DDPJobDefinition

First, we begin by authenticating using the SDK.

In [ ]:
# Create authentication object for oc user permissions
auth = TokenAuthentication(
    token = "XXXXX",
    server = "XXXXX",
    skip_tls=True
)
auth.login()

Now that we are logged in, we can directly submit our batch job (model training on two workers with four gpus each) to MCAD via torchx.

In [6]:
jobdef = DDPJobDefinition(
    name="mnistjob",
    script="mnist.py",
    scheduler_args={"namespace": "default"},
    j="1x1",
    gpu=0,
    cpu=1,
    memMB=8000,
    image="quay.io/michaelclifford/mnist-test:latest"
)
job = jobdef.submit()

Now we can go ahead and look at the status and logs of our batch job.

In [7]:
job.status()

/home/meyceoz/Documents/sdktest/lib64/python3.8/site-packages/torchx/schedulers/kubernetes_mcad_scheduler.py:1047: UserWarning: Warning - MCAD does not report individual replica statuses, but overall task status. Replica id  may not match status
  warnings.warn(msg)


AppStatus:
  msg: <NONE>
  num_restarts: -1
  roles:
  - replicas:
    - hostname: ''
      id: 0
      role: mnist
      state: !!python/object/apply:torchx.specs.api.AppState
      - 3
      structured_error_msg: <NONE>
    role: mnist
  state: RUNNING (3)
  structured_error_msg: <NONE>
  ui_url: null

In [19]:
print(job.logs())

2023-04-04T21:07:18.216609937Z INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:
2023-04-04T21:07:18.216609937Z   entrypoint       : mnist.py
2023-04-04T21:07:18.216609937Z   min_nodes        : 1
2023-04-04T21:07:18.216609937Z   max_nodes        : 1
2023-04-04T21:07:18.216609937Z   nproc_per_node   : 1
2023-04-04T21:07:18.216609937Z   run_id           : mnistjob-d6hz7lmsvx4scd
2023-04-04T21:07:18.216609937Z   rdzv_backend     : static
2023-04-04T21:07:18.216609937Z   rdzv_endpoint    : localhost:49782
2023-04-04T21:07:18.216609937Z   rdzv_configs     : {'rank': 0, 'timeout': 900}
2023-04-04T21:07:18.216609937Z   max_restarts     : 0
2023-04-04T21:07:18.216609937Z   monitor_interval : 5
2023-04-04T21:07:18.216609937Z   log_dir          : None
2023-04-04T21:07:18.216609937Z   metrics_cfg      : {}
2023-04-04T21:07:18.216609937Z 
2023-04-04T21:07:18.217434715Z INFO:torch.distributed.elastic.agent.server.local_elastic_agent:log directory set to: /tmp/torche

Finally, we can remove the job once complete and release/terminate the associated resources, bringing everything back to the way it was before job submission.

In [20]:
job.cancel()

In [ ]:
auth.logout()